In [7]:
"""
README: City-Specific YOLO Dataset Splitter

This script creates separate datasets for each city in a multi-city YOLO dataset.
- For each city, creates a new dataset directory with symlinks to the original 'train' images (full),
  and filtered symlinks for 'valid' and 'test' splits based on city-specific filename keywords.
- Updates and writes a new data.yaml for each city with correct split paths.

How to use:
- Set 'original_dataset' and 'target_base' paths as needed.
- Define your 'city_keywords' according to your dataset's city naming conventions.
- Run this script; one subfolder per city will be created, each with a ready-to-use data.yaml.

Author: Bahadir Akin Akgul
Date: 13.07.2025
"""

import os
import shutil
from pathlib import Path
import yaml

# Original dataset root
original_dataset = Path('/PATH/TO/original-dataset-root')

# Splits and data.yaml
train_dir = original_dataset / 'train'
valid_dir = original_dataset / 'valid'
test_dir = original_dataset / 'test'
data_yaml_path = original_dataset / 'data.yaml'

# City keyword filter (edit as needed)
city_keywords = {
    'istanbul': ['libadiye', 'levent', 'taksim', 'ciragan', 'barbaros', 'dolmabahce', 'bagdat', 'muallim', 'katar'],
    'paris': ['paris-champs'],
    'munich': ['munih'],
    'marseille': ['marsilya']
}

# Output base directory
target_base = Path('/PATH/TO/output-dataset-root')

for city, keywords in city_keywords.items():
    print(f"Creating dataset for {city.upper()}...")

    # City-specific dataset path
    city_dataset_dir = target_base / f'dataset-city-{city}'

    # Remove old city dataset if exists
    if city_dataset_dir.exists():
        print(f"Removing old directory: {city_dataset_dir}")
        shutil.rmtree(city_dataset_dir)

    # Symlink full train/ directory
    src_train = train_dir
    dst_train = city_dataset_dir / 'train'
    dst_train.parent.mkdir(parents=True, exist_ok=True)
    if not dst_train.exists():
        os.symlink(src_train, dst_train)
    print(f"Train folder symlinked.")

    # Filtered valid/ symlinks
    city_valid_images_dir = city_dataset_dir / 'valid' / 'images'
    city_valid_labels_dir = city_dataset_dir / 'valid' / 'labels'
    city_valid_images_dir.mkdir(parents=True, exist_ok=True)
    city_valid_labels_dir.mkdir(parents=True, exist_ok=True)

    for img_path in (valid_dir / 'images').glob('*.*'):
        img_name = img_path.name.lower()
        if any(keyword in img_name for keyword in keywords):
            dst_img = city_valid_images_dir / img_path.name
            if not dst_img.exists():
                os.symlink(img_path, dst_img)

            label_path = valid_dir / 'labels' / (img_path.stem + '.txt')
            if label_path.exists():
                dst_lbl = city_valid_labels_dir / label_path.name
                if not dst_lbl.exists():
                    os.symlink(label_path, dst_lbl)
            else:
                print(f"Label not found (valid): {label_path}")

    print(f"Valid folder for {city} ready.")

    # Filtered test/ symlinks
    city_test_images_dir = city_dataset_dir / 'test' / 'images'
    city_test_labels_dir = city_dataset_dir / 'test' / 'labels'
    city_test_images_dir.mkdir(parents=True, exist_ok=True)
    city_test_labels_dir.mkdir(parents=True, exist_ok=True)

    for img_path in (test_dir / 'images').glob('*.*'):
        img_name = img_path.name.lower()
        if any(keyword in img_name for keyword in keywords):
            dst_img = city_test_images_dir / img_path.name
            if not dst_img.exists():
                os.symlink(img_path, dst_img)

            label_path = test_dir / 'labels' / (img_path.stem + '.txt')
            if label_path.exists():
                dst_lbl = city_test_labels_dir / label_path.name
                if not dst_lbl.exists():
                    os.symlink(label_path, dst_lbl)
            else:
                print(f"Label not found (test): {label_path}")

    print(f"Test folder for {city} ready.")

    # Load and update data.yaml
    with open(data_yaml_path, 'r') as f:
        data_yaml = yaml.safe_load(f)

    # Update paths for the city's split
    data_yaml['train'] = str(dst_train / 'images')
    data_yaml['val'] = str(city_dataset_dir / 'valid' / 'images')
    data_yaml['test'] = str(city_dataset_dir / 'test' / 'images')

    # Save new data.yaml
    city_yaml_path = city_dataset_dir / 'data.yaml'
    with open(city_yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)

    print(f"data.yaml for {city} saved.\n")

print("All city datasets created successfully!")



🔵 ISTANBUL için dataset oluşturuluyor...
🧹 /truba/home/baakgul/roadtr-14032025-istanbul siliniyor...
✅ train klasörü symlink yapıldı.
✅ valid klasörü istanbul için hazır.
✅ test klasörü istanbul için hazır.
✅ data.yaml istanbul için kaydedildi.

🔵 PARIS için dataset oluşturuluyor...
🧹 /truba/home/baakgul/roadtr-14032025-paris siliniyor...
✅ train klasörü symlink yapıldı.
✅ valid klasörü paris için hazır.
✅ test klasörü paris için hazır.
✅ data.yaml paris için kaydedildi.

🔵 MUNIH için dataset oluşturuluyor...
🧹 /truba/home/baakgul/roadtr-14032025-munih siliniyor...
✅ train klasörü symlink yapıldı.
✅ valid klasörü munih için hazır.
✅ test klasörü munih için hazır.
✅ data.yaml munih için kaydedildi.

🔵 MARSILYA için dataset oluşturuluyor...
🧹 /truba/home/baakgul/roadtr-14032025-marsilya siliniyor...
✅ train klasörü symlink yapıldı.
✅ valid klasörü marsilya için hazır.
✅ test klasörü marsilya için hazır.
✅ data.yaml marsilya için kaydedildi.


🏁 Tüm şehirler için datasetler başarıyla oluş